In [1]:
import time
import numpy as np
import pandas as pd
import urllib.request 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
listings_data = pd.read_csv('./data/preprocessed_listings.csv')
listings_data.head(2)

C:\Users\Hossam Aboouf\AppData\Local\Temp\ipykernel_10988\3035141658.py:1: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  listings_data = pd.read_csv('./data/preprocessed_listings.csv')


,Unnamed: 0,id,listing_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,images_names,images_No
0,1,5136,https://www.airbnb.com/rooms/5136,100,100,0,1.0,5.0,1,1,...,4.75,4.75,1,1,1,0,0,0.03,"5136_0.jpg,5136_1.jpg,5136_2.jpg,5136_3.jpg,51...",5
1,2,6848,https://www.airbnb.com/rooms/6848,100,100,1,1.0,1.0,1,1,...,4.68,4.57,0,1,1,0,0,1.05,"6848_0.jpg,6848_1.jpg,6848_2.jpg,6848_3.jpg,68...",5


In [3]:
listings_data.shape

(19142, 48)

In [4]:
len(listings_data['id'].unique())

19142

In [5]:
new_ids = listings_data['listing_url'].apply(lambda item: int(item.split('/')[-1]))

ids_list = new_ids
for i, id in enumerate(ids_list):
    if id in ids_list[i+1 : ]:
        print(id)

5136
6848
6872
6990
7097
7801
8490
9357
10452
12192
12937
12940
13808
14314
15341
15385
16580
16821
16974
18728


In [ ]:
listings_data['listing_url']

In [6]:
listings_data[['price']].isna().sum()

price    0
dtype: int64

In [ ]:
def get_url_of_all_img_window(url):
    new_url = url
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 20)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    dialog = soup.select('.d1esrtf4[role="dialog"]')
    # print(dialog)
    WebDriverWait(driver, 20)

    if dialog :
        cls_button = driver.find_element(By.CSS_SELECTOR, ".d1esrtf4[role='dialog'] button.l1ovpqvx")
        cls_button.click()
        WebDriverWait(driver, 20)

    if "page not found" not in str.lower(soup.find("head").text):  
        show_all_button = driver.find_element(By.XPATH, "//div[contains(text(), 'Show all photos')]")
        if show_all_button:
            show_all_button.click()
            WebDriverWait(driver, 20)
            driver.switch_to.window(driver.window_handles[0])
            new_url = driver.current_url
    else:
        print(f'Page with url ({url}) is 404 page.')

    return new_url

In [7]:
def save_image(img_url, img_name):
    urllib.request.urlretrieve(img_url, f'./images/{img_name}') 

In [8]:
listings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19142 entries, 0 to 19141
Data columns (total 48 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    19142 non-null  int64  
 1   id                                            19142 non-null  int64  
 2   listing_url                                   19142 non-null  object 
 3   host_response_rate                            19142 non-null  int64  
 4   host_acceptance_rate                          19142 non-null  int64  
 5   host_is_superhost                             19142 non-null  int64  
 6   host_listings_count                           19142 non-null  float64
 7   host_total_listings_count                     19142 non-null  float64
 8   host_has_profile_pic                          19142 non-null  int64  
 9   host_identity_verified                        19142 non-null 

In [10]:
for row_index, row in listings_data.iloc[1325:].iterrows():
    base_url = row['listing_url']
    print(f'working on url: ({base_url})') 
    images_No = 0  
    id = int(base_url.split('/')[-1]) # extract the id form url
    driver = webdriver.Chrome()
    driver.get(base_url)
    WebDriverWait(driver, 10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')    
    dialog = soup.select('.d1esrtf4[role="dialog"]')

    if dialog :
        cls_button = driver.find_element(By.CSS_SELECTOR, ".d1esrtf4[role='dialog'] button.l1ovpqvx")
        cls_button.click()
        WebDriverWait(driver, 10)
    
    try:
        images = driver.find_elements(By.CSS_SELECTOR, 'div.d1l1iq7v picture .itu7ddv')
        images = list(map(lambda img: img.get_attribute('src'), images))
        images = np.unique(np.array(images))
        images_No = len(images)
        all_imgs_names = []

        for i, img_url in enumerate(images):
            all_imgs_names.append(f'{id}_{i}.jpg')
            save_image(img_url, all_imgs_names[i])
            if i >= 4:
                images_No = 5
                break
        print('images No.: ', images_No)
    except:
        all_imgs_names = None

    driver.quit()
    row['images_names'] = ','.join(all_imgs_names)
    row['images_No'] = images_No
    listings_data.iloc[row_index] = row
    listings_data.to_csv('./data/preprocessed_listings.csv', encoding='utf-8', index=False)

working on url: (https://www.airbnb.com/rooms/919830)
images No.:  5
working on url: (https://www.airbnb.com/rooms/927408)
images No.:  5
working on url: (https://www.airbnb.com/rooms/936218)
images No.:  5
working on url: (https://www.airbnb.com/rooms/939657)
images No.:  5
working on url: (https://www.airbnb.com/rooms/940115)
images No.:  5
working on url: (https://www.airbnb.com/rooms/940116)
images No.:  5
working on url: (https://www.airbnb.com/rooms/945297)
images No.:  5
working on url: (https://www.airbnb.com/rooms/951365)
images No.:  5
working on url: (https://www.airbnb.com/rooms/951439)
images No.:  5
working on url: (https://www.airbnb.com/rooms/956412)
images No.:  5
working on url: (https://www.airbnb.com/rooms/971233)
images No.:  5
working on url: (https://www.airbnb.com/rooms/973535)
images No.:  5
working on url: (https://www.airbnb.com/rooms/975250)
images No.:  5
working on url: (https://www.airbnb.com/rooms/978386)
images No.:  5
working on url: (https://www.airbn

WebDriverException: Message: unknown error: net::ERR_CONNECTION_RESET
  (Session info: chrome=127.0.6533.72)
Stacktrace:
	GetHandleVerifier [0x00007FF789959632+30946]
	(No symbol) [0x00007FF78990E3C9]
	(No symbol) [0x00007FF789806FDA]
	(No symbol) [0x00007FF7897FF0B5]
	(No symbol) [0x00007FF7897F01E9]
	(No symbol) [0x00007FF7897F1D62]
	(No symbol) [0x00007FF7897F04CE]
	(No symbol) [0x00007FF7897EFD41]
	(No symbol) [0x00007FF7897EFC80]
	(No symbol) [0x00007FF7897EDE84]
	(No symbol) [0x00007FF7897EE29C]
	(No symbol) [0x00007FF78980A041]
	(No symbol) [0x00007FF78989BA2E]
	(No symbol) [0x00007FF78987CA6A]
	(No symbol) [0x00007FF78989B041]
	(No symbol) [0x00007FF78987C813]
	(No symbol) [0x00007FF78984A6E5]
	(No symbol) [0x00007FF78984B021]
	GetHandleVerifier [0x00007FF789A8F83D+1301229]
	GetHandleVerifier [0x00007FF789A9BDB7+1351783]
	GetHandleVerifier [0x00007FF789A92A03+1313971]
	GetHandleVerifier [0x00007FF78998DD06+245686]
	(No symbol) [0x00007FF78991758F]
	(No symbol) [0x00007FF7899138

In [16]:
# 528485
listings_data.head()


,Unnamed: 0,id,listing_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,images_names,images_No
0,1,5136,https://www.airbnb.com/rooms/5136,100,100,0,1.0,5.0,1,1,...,4.75,4.75,1,1,1,0,0,0.03,"[5136_0.jpg, 5136_1.jpg, 5136_2.jpg, 5136_3.jp...",5
1,2,6848,https://www.airbnb.com/rooms/6848,100,100,1,1.0,1.0,1,1,...,4.68,4.57,0,1,1,0,0,1.05,"[6848_0.jpg, 6848_1.jpg, 6848_2.jpg, 6848_3.jp...",5
2,3,6872,https://www.airbnb.com/rooms/6872,50,0,0,2.0,2.0,1,1,...,5.00,5.00,0,2,0,2,0,0.04,NaN,0
3,4,6990,https://www.airbnb.com/rooms/6990,100,100,1,1.0,6.0,1,1,...,4.85,4.84,0,1,0,1,0,1.38,NaN,0
4,6,7097,https://www.airbnb.com/rooms/7097,100,100,1,2.0,2.0,1,1,...,4.94,4.82,1,2,0,2,0,2.12,NaN,0


In [ ]:
listings_data.tail()

In [26]:
for row_index, row in listings_data.iloc[249:].iterrows():
    print(row_index)
    print(row['listing_url'])
    break

249
https://www.airbnb.com/rooms/528485
